<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 5. 评估

恭喜您完成今天的课程！希望这是一次有趣的旅程，收获了一些新技能。现在是时候把这些技能拿出来考验一下了。挑战来了：假设我们有一个分类模型，使用 LiDAR 数据来分类球体和立方体。相比于 RGB 摄像头，LiDAR 传感器并不那么容易获得，所以想将模型转换为能分类 RGB 图像的。由于使用了 [NVIDIA Omniverse](https://www.nvidia.com/en-us/omniverse/) 来生成 LiDAR 和 RGB 数据对，可以用这些数据创建一个对比预训练模型。由于 CLIP 已经被占用，我们将这个模型称为 `CILP`，代表“对比图像 LiDAR 预训练”。 

## 5.1 设置

### 整体步骤：
1. 加载已训练好的模型 lidar_cnn
2. 用 lidar 和 rgb 训练 clip，再用这个 clip 嵌入 RGB，得到嵌入向量，经过 projector 转换后，得到新的 lidar_cnn
3. 训练 projector，实现跨模态映射
4. 构建新的图像分类模型 RGB2LiDARClassifier

开始吧。以下是评估中使用的库。

In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from assessment import assesment_utils
from assessment.assesment_utils import Classifier
import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

### 5.1.1 模型

接下来，让我们加载分类模型并命名为 `lidar_cnn`。如果我们抽出一点时间查看一下 [assement_utils](assessment/assesment_utils.py)，可以看到用于构建模型的 `Classifier` 类。请注意 `get_embs` 方法，我们将使用它来构建我们的跨模态映射器。

In [2]:
lidar_cnn = Classifier(1).to(device)
lidar_cnn.load_state_dict(torch.load("assessment/lidar_cnn.pt", weights_only=True))
# Do not unfreeze. Otherwise, it would be difficult to pass the assessment.
for param in lidar_cnn.parameters():
    lidar_cnn.requires_grad = False
lidar_cnn.eval()

Classifier(
  (embedder): Sequential(
    (0): Conv2d(1, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(100, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(200, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3200, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)

### 5.1.2 数据集

以下是此评估中将使用的数据集。它与前几次实验中使用的数据集类似，但注意 `self.classes`。不同于第一节课的位置预测，这节我们将判断所评估的 RGB 或 LiDAR 是否包含 `cube` 或 `sphere`。

In [3]:
IMG_SIZE = 64
img_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),  # Scales data into [0,1]
])

class MyDataset(Dataset):
    def __init__(self, root_dir, start_idx, stop_idx):
        self.classes = ["cubes", "spheres"]
        self.root_dir = root_dir
        self.rgb = []
        self.lidar = []
        self.class_idxs = []

        for class_idx, class_name in enumerate(self.classes):
            for idx in range(start_idx, stop_idx):
                file_number = "{:04d}".format(idx)
                rbg_img = Image.open(self.root_dir + class_name + "/rgb/" + file_number + ".png")
                rbg_img = img_transforms(rbg_img).to(device)
                self.rgb.append(rbg_img)
    
                lidar_depth = np.load(self.root_dir + class_name + "/lidar/" + file_number + ".npy")
                lidar_depth = torch.from_numpy(lidar_depth[None, :, :]).to(torch.float32).to(device)
                self.lidar.append(lidar_depth)

                self.class_idxs.append(torch.tensor(class_idx, dtype=torch.float32)[None].to(device))

    def __len__(self):
        return len(self.class_idxs)

    def __getitem__(self, idx):
        rbg_img = self.rgb[idx]
        lidar_depth = self.lidar[idx]
        class_idx = self.class_idxs[idx]
        return rbg_img, lidar_depth, class_idx

这些数据可以在 `/data/assessment` 文件夹中找到。这里是一个立方体的示例。虽然图像比较小，但细节足以让我们的模型进行区分。

<center><img src="data/assessment/cubes/rgb/0002.png" /></center>

接下来将数据加载到 `DataLoader` 中。我们会为验证留出一些批次（`VALID_BATCHES`）。其余的数据将用于训练。每种立方体和球体类别各有 `9999` 张图像，所以这里将 N 乘 2，以反映合并后的数据集。

In [4]:
BATCH_SIZE = 32
VALID_BATCHES = 10
N = 9999

valid_N = VALID_BATCHES*BATCH_SIZE
train_N = N - valid_N

train_data = MyDataset("data/assessment/", 0, train_N)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
valid_data = MyDataset("data/assessment/", train_N, N)
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

N *= 2
valid_N *= 2
train_N *= 2

<a id="contrastive-pre-training"></a>

## 5.2 对比预训练

在创建跨模态映射模型之前，最好能有一个对 RGB 图像进行嵌入的方式。让我们利用数据，创建一个对比预训练模型。首先需要一个卷积模型，下面是推荐的架构。

In [5]:
CILP_EMB_SIZE = 200

class Embedder(nn.Module):
    def __init__(self, in_ch, emb_size=CILP_EMB_SIZE):
        super().__init__()
        kernel_size = 3
        stride = 1
        padding = 1

        # Convolution
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, 50, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(50, 100, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(100, 200, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(200, 200, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )

        # Embeddings
        self.dense_emb = nn.Sequential(
            nn.Linear(200 * 4 * 4, 100),
            nn.ReLU(),
            nn.Linear(100, emb_size)
        )

    def forward(self, x):
        conv = self.conv(x)
        emb = self.dense_emb(conv)
        return F.normalize(emb)

RGB 数据有 `4` 个通道，而我们的 LiDAR 数据有 `1` 个通道。分别初始化这些嵌入模型。

In [6]:
img_embedder = Embedder(4).to(device)
lidar_embedder = Embedder(1).to(device)

有嵌入模型了，现在把它们组合成一个 `ContrastivePretraining` 模型。

**TODO**: 下面的 `ContrastivePretraining` 类几乎完成，但还有一些 `FIXME`。请替换这些 `FIXME`，以使模型能工作。您可以随时查看 notebook [02b_Contrastive_Pretraining.ipynb](02b_Contrastive_Pretraining.ipynb) 获取提示。

混合模型需要加载两个模型

In [7]:
class ContrastivePretraining(nn.Module):
    def __init__(self):
        super().__init__()
        self.img_embedder = img_embedder
        self.lidar_embedder = lidar_embedder
        self.cos = nn.CosineSimilarity()

    def forward(self, rgb_imgs, lidar_depths):
        img_emb = self.img_embedder(rgb_imgs)
        lidar_emb = self.lidar_embedder(lidar_depths)

        repeated_img_emb = img_emb.repeat_interleave(len(img_emb), dim=0)
        repeated_lidar_emb = lidar_emb.repeat(len(lidar_emb), 1)

        similarity = self.cos(repeated_img_emb, repeated_lidar_emb)
        similarity = torch.unflatten(similarity, 0, (BATCH_SIZE, BATCH_SIZE))
        similarity = (similarity + 1) / 2

        logits_per_img = similarity
        logits_per_lidar = similarity.T
        return logits_per_img, logits_per_lidar

是时候验证一下模型了！先初始化。

In [8]:
CILP_model = ContrastivePretraining().to(device)
optimizer = Adam(CILP_model.parameters(), lr=0.0001)
loss_img = nn.CrossEntropyLoss()
loss_lidar = nn.CrossEntropyLoss()
ground_truth = torch.arange(BATCH_SIZE, dtype=torch.long).to(device)
epochs = 3

训练模型之前，应该定义一个损失函数来指导模型学习。

**TODO**: 下面的 `get_CILP_loss` 函数也几乎完成。您还记得计算损失的公式吗？请替换下面的 `FIXME`。

由前面的代码加上变量倒推得到

In [11]:
def get_CILP_loss(batch):
    rbg_img, lidar_depth, class_idx = batch
    logits_per_img, logits_per_lidar = CILP_model(rbg_img, lidar_depth)
    total_loss = (loss_img(logits_per_img, ground_truth) + loss_lidar(logits_per_lidar, ground_truth))/2
    return total_loss, logits_per_img

下面就该开始训练了。如果上面的 `TODO` 都正确完成，损失应该在 `3.2` 以下。看看对角线的值接近 `1` 吗？

In [12]:
for epoch in range(epochs):
    CILP_model.train()
    train_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        loss, logits_per_img = get_CILP_loss(batch)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    assesment_utils.print_CILP_results(epoch, train_loss/step, logits_per_img, is_train=True)

    CILP_model.eval()
    valid_loss = 0
    for step, batch in enumerate(valid_dataloader):
        loss, logits_per_img = get_CILP_loss(batch)
        valid_loss += loss.item()
    assesment_utils.print_CILP_results(epoch, valid_loss/step, logits_per_img, is_train=False)

Epoch 0
Train Loss: 3.0814396334722467 
Similarity:
tensor([[0.9961, 0.8826, 0.9635,  ..., 0.4457, 0.9774, 0.3969],
        [0.8954, 0.9949, 0.9433,  ..., 0.2372, 0.9658, 0.2058],
        [0.9508, 0.8960, 0.9951,  ..., 0.3770, 0.9709, 0.3361],
        ...,
        [0.4260, 0.1997, 0.3297,  ..., 0.9967, 0.3119, 0.9961],
        [0.9024, 0.9299, 0.9854,  ..., 0.2893, 0.9593, 0.2568],
        [0.3884, 0.1798, 0.3019,  ..., 0.9917, 0.2811, 0.9962]],
       device='cuda:0', grad_fn=<DivBackward0>)
Valid Loss: 3.1908093251680074 
Similarity:
tensor([[0.9883, 0.7652, 0.4555,  ..., 0.5229, 0.5947, 0.9643],
        [0.8368, 0.9938, 0.3124,  ..., 0.2135, 0.2656, 0.8444],
        [0.3505, 0.2415, 0.9936,  ..., 0.7209, 0.6181, 0.2714],
        ...,
        [0.4660, 0.2214, 0.6955,  ..., 0.9977, 0.9824, 0.4480],
        [0.5447, 0.2682, 0.5989,  ..., 0.9821, 0.9969, 0.5403],
        [0.9913, 0.8119, 0.3122,  ..., 0.4702, 0.5682, 0.9970]],
       device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1
Trai

完成后，请冻结模型。后面将使用这个模型和跨模型映射模型进行评估。如果跨模型映射训练期间修改了这个模型的话，可能就没法通过评估了！

In [13]:
for param in CILP_model.parameters():
    CILP_model.requires_grad = False

## 5.3 跨模态映射

现在有嵌入图像数据的方法了，接下来就来做跨模态映射吧。

**TODO**: 让我们直接开始，创建映射器。模型的输入维度应该是什么，输出维度又应该是什么呢？关于第一个 `FIXME` 的提示可以在 `Embedder` 类的[#5.2 对比预训练](#contrastive-pre-training)找到。第二个 `FIXME` 的提示可以在 `assessment/assesment_utils.py` 文件的 `Classifier` 类中找到。第二个 `FIXME` 的维度应该和 `get_embs` 函数的输出大小一致。

线性代数知识：矩阵乘法要求第一个矩阵的列数必须等于第二个矩阵的行数
报错提示前者为 200
由代码得到后者答案为 3200

In [ ]:
self.classifier = nn.Sequential(
            nn.Linear(200 * 4 * 4, 100),
            nn.ReLU(),
            nn.Linear(100, n_classes)
        )

In [ ]:
projector = nn.Sequential(
    nn.Linear(200, 1000),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500, 3200)
).to(device)

In [ ]:
# 输出
Classifier(
  (embedder): Sequential(
    (0): Conv2d(1, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(100, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(200, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3200, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [ ]:
projector = nn.Sequential(
    nn.Linear(clip_emb_size[0]*patches, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, vgg_shape[0])
).to(device)

输入是 clip 需要的，输出是 vgg 需要的，中间自己定
转为新向量后作为分类头送去训练
clip -> projector -> classifier

接下来，定义训练 `projector` 的损失函数。

**TODO**: 用于估计映射嵌入的损失函数是什么？请替换下面的 `FIXME`。可以查看 notebook [03a_Projection.ipynb](03a_Projection.ipynb) 的 3.2 部分获得提示。

In [ ]:
def get_projector_loss(model, batch):
    rbg_img, lidar_depth, class_idx = batch
    imb_embs = CILP_model.img_embedder(rbg_img)
    lidar_emb = lidar_cnn.get_embs(lidar_depth)
    pred_lidar_embs = model(imb_embs)
    return nn.FIXME()(pred_lidar_embs, lidar_emb)

In [ ]:
def get_projector_loss(model, batch):
    imgs, texts, _ = batch
    imb_embs = flower_classifier.get_img_embs(imgs)

    text_encodings = get_clip_encodings(texts)
    pred_img_embs = model(text_encodings).to(device)
    return nn.MSELoss()(pred_img_embs, imb_embs), 0

训练 `projector` 会花费一些时间，最终验证损失应该达到 2 左右。

In [ ]:
epochs = 40
optimizer = torch.optim.Adam(projector.parameters())
assesment_utils.train_model(projector, optimizer, get_projector_loss, epochs, train_dataloader, valid_dataloader)

是时候把它们合一起了。创建一个新模型 `RGB2LiDARClassifier`，用上映射器和预训练的 `lidar_cnn` 模型。

**TODO**: 请填上 `FIXME`。现在应该用 `CILP_model` 的哪个 `embedder`？

In [ ]:
class RGB2LiDARClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.projector = projector
        self.FIXME = CILP_model.FIXME_embedder
        self.shape_classifier = lidar_cnn
    
    def forward(self, imgs):
        img_encodings = self.img_embedder(imgs)
        proj_lidar_embs = self.projector(img_encodings)
        return self.shape_classifier(data_embs=proj_lidar_embs)

In [ ]:
my_classifier = RGB2LiDARClassifier()

在训练这个模型之前，先看看它开箱即用的效果。我们创建一个 `get_correct` 函数，用于计算正确分类的数量。

In [ ]:
def get_correct(output, y):
    zero_tensor = torch.tensor([0]).to(device)
    pred = torch.gt(output, zero_tensor)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct

接下来用一个 `get_valid_metrics` 函数，计算模型在验证数集的准确率。如果做对了，准确率应该超过 `.70`，也就是 70%。

In [ ]:
def get_valid_metrics():
    my_classifier.eval()
    correct = 0
    batch_correct = 0
    for step, batch in enumerate(valid_dataloader):
        rbg_img, _, class_idx = batch
        output = my_classifier(rbg_img)
        loss = nn.BCEWithLogitsLoss()(output, class_idx)
        batch_correct = get_correct(output, class_idx)
        correct += batch_correct
    print(f"Valid Loss: {loss.item():2.4f} | Accuracy {correct/valid_N:2.4f}")

get_valid_metrics()

最后，让我们微调完成的模型。因为 `CILP` 和 `lidar_cnn` 都被冻结，所以这只会改变模型的 `projector` 部分。模型的验证准确率也应该能超过 `.95` 也就是 95%。

In [ ]:
epochs = 5
optimizer = torch.optim.Adam(my_classifier.parameters())

my_classifier.train()
for epoch in range(epochs):
    correct = 0
    batch_correct = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        rbg_img, _, class_idx = batch
        output = my_classifier(rbg_img)
        loss = nn.BCEWithLogitsLoss()(output, class_idx)
        batch_correct = get_correct(output, class_idx)
        correct += batch_correct
        loss.backward()
        optimizer.step()
    print(f"Train Loss: {loss.item():2.4f} | Accuracy {correct/train_N:2.4f}")
    get_valid_metrics()

## 5.4 运行评估

为了评估您的模型，请运行以下两个单元格。评估满分为 10 分：

* 确保 CILP 的验证损失低于 `3.2`（5 分）
* 确保 `projector` 可以与 `lidar_cnn` 一起使用，准确分类图像。如果准确率超过 `.95`，将测试五个批次的图像。（每个批次 1 分，总共 5 分）

您需要获得 10 分中的 9 分才能通过评估。祝您好运！

请在下面提交您的 `CILP_model` 和 `projector`。如果这些模型的名称已经更改，请相应更新。

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(CILP_model, projector)

## 6.7 生成证书

如果您通过了评估，请返回课程页面并点击 "ASSESS TASK" 按钮，这将为您生成课程证书。